In [ ]:
import pure_pursuit as pp
import matplotlib.pyplot as plt
import numpy as np

PURSUER_COLOR="tab:red"
EVADER_COLOR="tab:blue"
SUCCESS_COLOR="tab:green"
FAIL_COLOR="tab:orange"

MAX_DISTANCE = 10

def plot_engagement(d_left, d_right, lod_left, lod_right, mu_left, mu_right, angle_between):
    # do the math
    ha = angle_between/2

    th_l = pp.optimal_evader_heading(d_left/d_right, lod_left, lod_right, mu_left, mu_right, angle_between, 10)
    th_r = angle_between - th_l - np.pi

    l_left = lod_left * d_left
    l_right = lod_right * d_right

    r_min_l = pp.r_min(th_l, mu_left)
    r_min_r = pp.r_min(th_r, mu_right)

    margin_l = r_min_l * d_left - l_left
    margin_r = r_min_r * d_right - l_right
    valid = np.allclose(margin_l, margin_r, atol=0.0001)

    th_adj = -th_l + angle_between/2 - np.pi/2
    d_max = max(d_left, d_right)

    success = r_min_l * d_left > l_left and r_min_r * d_right > l_right

    # put some stuff on the screen
    if valid:
        print(f"Optimal: margin to left capture = margin to right capture")
        print(f"({margin_l} == {margin_r})\n")
    else:
        print(f"!NOT OPTIMAL!: {margin_l} != {margin_r}\n")

    if success:
        print("Evader escapes")
    else:
        print("Evader is captured")

    # draw actors
    fig, ax = plt.subplots()
    ax.scatter([0], [0], c=EVADER_COLOR)

    left_pos = [np.cos(ha)*d_left, np.sin(ha)*d_left]
    right_pos = [np.cos(-ha) * d_right, np.sin(-ha) * d_right]
    ax.scatter(
        [left_pos[0], right_pos[0]], 
        [left_pos[1], right_pos[1]],
        c=PURSUER_COLOR
    )

    # draw trajectories
    v_e = MAX_DISTANCE / 4
    v_l = v_e / mu_left
    v_r = v_e / mu_right

    ax.arrow(
        0, 0, np.cos(th_adj) * v_e, np.sin(th_adj) * v_e,
        head_width=0.2,
        color=SUCCESS_COLOR if success else FAIL_COLOR
    )

    ax.arrow(
        np.cos(ha) * d_left, np.sin(ha) * d_left,
        -v_l * np.cos(ha), -v_l * np.sin(ha),
        head_width=0.2, color=PURSUER_COLOR
    )

    ax.arrow(
        np.cos(-ha) * d_right, np.sin(-ha) * d_right,
        -v_r * np.cos(-ha), -v_r * np.sin(-ha),
        head_width=0.2, color=PURSUER_COLOR
    )

    # draw capture radius
    cap_l = plt.Circle(left_pos, l_left, color=PURSUER_COLOR, alpha=0.3)
    cap_r = plt.Circle(right_pos, l_right, color=PURSUER_COLOR, alpha=0.3)

    ax.add_patch(cap_l)
    ax.add_patch(cap_r)

    # other bookkeeping
    ax.set_aspect("equal")
    
    margin = 1
    size = MAX_DISTANCE+margin
    ax.set_xlim(-size, size)
    ax.set_ylim(-size, size)

    return ax

In [ ]:
import ipywidgets

# note: when mu is high enough, optimal angle is nan at high angle_between
ipywidgets.interact(
    plot_engagement,
    d_left=(0.1, MAX_DISTANCE),
    d_right=(0.1, MAX_DISTANCE),
    lod_left=(0.1, 1.0, 0.01),
    lod_right=(0.1, 1.0, 0.01),
    mu_left=(1.1, 5.0),
    mu_right=(1.1, 5.0),
    angle_between=(0.001, np.pi)
)